In [41]:
# Importação das bibliotecas usadas no desenvolvimento do projeto
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFECV, RFE
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Importação dos conjutos de amostras
dados_treino = pd.read_csv("conjunto_de_treinamento.csv", delimiter=",", decimal=".")
dados_teste = pd.read_csv("conjunto_de_teste.csv", delimiter=",", decimal=".")

In [3]:
dados_teste.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha
0,20001,1,25,presencial,1,M,37,2,0,0,...,,N,,0,0.0,0.0,0.0,0.0,384.0,384.0
1,20002,1,10,internet,1,F,31,2,0,0,...,RJ,N,,0,9.0,5.0,NaN,NaN,275.0,275.0
2,20003,1,10,internet,1,F,18,2,0,0,...,RS,N,,0,9.0,2.0,NaN,NaN,948.0,948.0
3,20004,1,10,presencial,1,F,55,2,0,0,...,,N,,0,9.0,1.0,0.0,0.0,581.0,581.0
4,20005,1,10,presencial,1,F,55,1,0,0,...,,N,,0,0.0,1.0,0.0,0.0,573.0,573.0


In [4]:
dados_treino.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1


Como é possível observar nas 5 primeiras linhas dos dados de treino e teste, existem dados faltantes "NaN". Dado isto, quantifica-se a perda de dados ao simplesmente eliminar as linhas com NaN.

In [5]:
linha_treino_com_na = dados_treino.shape[0]
linha_treino_sem_na = dados_treino.dropna().shape[0]

print("Perda de %0.2f%% dos dados de treino ao retirar linhas com informações faltantes." %(100 - linha_treino_sem_na*100/linha_treino_com_na))

Perda de 65.12% dos dados de treino ao retirar linhas com informações faltantes.


A perda de 65.12% do dataset é algo péssimo para o modelo pois perde grande parte da representação do universo. Para prosseguir sem perder essas amostras, optou-se por substituir os NaN de forma a manter a característica das distribuições.

Para fazer essas substituições, primeiro fa-se o tratamento dos dados nos datasets de treino e teste. A seguir seguem as modificaçãoes feitas sobre os dados.

Primeiramente faz-se a análise do arquivo "dicionario_de_dados.xlsx" e segue-se as recomendações para remover as colunas com erro de preenchimento e colunas que visivelmente agregam pouco valor (por ter alta correlação com outra coluna. Ex. colunas de estados onde trabalha, habita e nasceu).

In [6]:
# forma_envio_solicitacao -> Abrir em 3 colunas (internet, correio e presencial)
# sexo -> Abrir em 3 colunas (masculino, feminino, nao informado). Se vazio colocar N
# grau_instrucao -> Remover
# estado_onde_nasceu -> Remover, possui alta correlação  com "estado_onde_reside"
# estado_onde_reside -> Transformar em regiões do país
# possui_telefone_residencial -> Transformar em 1 - Sim 0 - Nao
# possui_telefone_celular ->  Remover
# qtde_contas_bancarias_especiais -> Remover
# vinculo_formal_com_empresa -> Transformar em 1 - Sim 0 - Nao
# estado_onde_trabalha -> Inútil, apenas 0.2% está preenchido
# possui_telefone_trabalho -> Transformar em 1 - Sim 0 - Nao
# meses_no_trabalho -> Remover

# deleta as colunas desnecessárias/com dados corrompidos
dados_treino = dados_treino.drop(columns=['id_solicitante', 'grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho', 'estado_onde_trabalha',\
                                         'codigo_area_telefone_trabalho', 'codigo_area_telefone_residencial', 'estado_onde_nasceu'])
dados_teste = dados_teste.drop(columns=['id_solicitante', 'grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho', 'estado_onde_trabalha',\
                                         'codigo_area_telefone_trabalho', 'codigo_area_telefone_residencial', 'estado_onde_nasceu'])

In [7]:
# splita as colunas sexo e forma_envio_solicitacao em uma coluna para cada opção dentre as possíveis
# caso o valor de uma linha da coluna sexo seja NaN, troca por N - Não informado
valores = {'sexo':'N'}

dados_treino = dados_treino.fillna(value=valores)
dados_teste = dados_teste.fillna(value=valores)

dados_treino = pd.get_dummies(dados_treino, columns=['forma_envio_solicitacao', 'sexo'])
dados_teste = pd.get_dummies(dados_teste, columns=['forma_envio_solicitacao', 'sexo'])

# drop necessário para tirar uma coluna de bug
dados_treino = dados_treino.drop(columns=['sexo_ '])
dados_teste = dados_teste.drop(columns=['sexo_ '])

In [8]:
# Troca os Y e N por 1 e 0, respectivamente nas colunas dentro da lista abaixo
valores_a_trocar = ['possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho']

for coluna in valores_a_trocar:
    dados_treino[coluna] = dados_treino[coluna].map({'Y':1, 'N':0}, na_action=None)
    dados_teste[coluna] = dados_teste[coluna].map({'Y':1, 'N':0}, na_action=None)

dados_treino = pd.get_dummies(dados_treino, columns=valores_a_trocar)
dados_teste = pd.get_dummies(dados_teste, columns=valores_a_trocar)

In [9]:
# Troca os estados das colunas na lista abaixo pela região do estado e faz o get_dummie da coluna
valores_a_trocar = ['estado_onde_reside']

dicionario = {'AC':'NO', 'AP':'NO', 'AM':'NO', 'PA':'NO', 'RO':'NO', 'RR':'NO', 'TO':'NO',\
              'MA':'NE', 'PI':'NE', 'CE':'NE', 'RN':'NE', 'PB':'NE', 'PE':'NE', \
             'AL':'NE', 'SE':'NE', 'BA':'NE', 'MT':'CO', 'MS':'CO', 'GO':'CO',\
             'DF':'CO', 'RJ':'SD', 'MG':'SD', 'SP':'SD', 'ES':'SD', 'SC':'SU', 'PR':'SU', 'RS':'SU'}

for coluna in valores_a_trocar:
    dados_treino[coluna] = dados_treino[coluna].map(dicionario, na_action='NC')
    dados_teste[coluna] = dados_teste[coluna].map(dicionario, na_action='NC')
    
dados_treino = pd.get_dummies(dados_treino, columns=valores_a_trocar)
dados_teste = pd.get_dummies(dados_teste, columns=valores_a_trocar)

Após as tratativas nas últimas 4 células, finalmente tem-se dados com todas as linhas preenchidas. Ao longo das tratativas alguns dados foram transformados e/ou removidos, alterando de alguma forma na precisão do modelo.

Agora, com todos os dados transformados pode-se substituir os valores faltantes por alguma métrica.

In [10]:
# Substitui os dados NaN pela mediana da coluna
dados_teste = dados_teste.fillna(dados_teste.median())
dados_treino = dados_treino.fillna(dados_treino.median())

In [11]:
for coluna in dados_treino.columns:
    print(dados_treino[coluna])

0        1
1        1
2        1
3        1
4        7
        ..
19995    1
19996    1
19997    1
19998    1
19999    2
Name: produto_solicitado, Length: 20000, dtype: int64
0        10
1        25
2        20
3        20
4         1
         ..
19995    10
19996    20
19997    10
19998     5
19999    20
Name: dia_vencimento, Length: 20000, dtype: int64
0        1
1        1
2        1
3        1
4        1
        ..
19995    1
19996    1
19997    1
19998    1
19999    1
Name: tipo_endereco, Length: 20000, dtype: int64
0        85
1        38
2        37
3        37
4        51
         ..
19995    27
19996    26
19997    63
19998    84
19999    53
Name: idade, Length: 20000, dtype: int64
0        2
1        1
2        2
3        1
4        1
        ..
19995    2
19996    2
19997    2
19998    1
19999    1
Name: estado_civil, Length: 20000, dtype: int64
0        0
1        0
2        0
3        1
4        3
        ..
19995    0
19996    1
19997    0
19998    0
19999    0
Name: qtde

In [51]:
#separacao do conjunto de treino em alvo e features e em subconjunto de teste e treino
dados_treino = dados_treino.sample(frac=1)

X = dados_treino.drop(columns=['inadimplente'])
Y = dados_treino['inadimplente']


standard_scaler = MinMaxScaler()
standard_scaler = standard_scaler.fit(X)
X = standard_scaler.transform(X)

x_treino = X[:16000]
y_treino = Y[:16000]

x_teste = X[16000:]
y_teste = Y[16000:]

In [43]:
# Faz o Scaling das colunas
# idade, dia_vencimento, qtde_dependentes, meses_na_residencia, renda_mensal_regular, renda_extra, valor_patrimonio_pessoal, local_onde_reside, local_onde_trabalha

standard_scaler = MinMaxScaler()
standard_scaler = standard_scaler.fit(x_treino)
x_treino = standard_scaler.transform(x_treino)



In [52]:
x_treino

array([[0.        , 0.58333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.16666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.58333333, 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.        , 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.375     , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.375     , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [53]:
# Decisão com SVM, RF e KNN 1 contra todos
print("  K   |  %   ")

for numero_kneighbors in range(1800, 1860, 10):
    classificador = KNeighborsClassifier(n_neighbors=numero_kneighbors, weights='uniform')
    
    classificador = classificador.fit(x_treino, y_treino)
    y_resposta_teste = classificador.predict(x_teste)
    y_resposta_treino = classificador.predict(x_treino)
    
    total = len(y_teste)
    acertos_teste = sum(y_resposta_teste==y_teste)
    acertos_treino = sum(y_resposta_treino==y_treino)
    
    acuracia_teste = acertos_teste / total
    acuracia_treino = acertos_treino/len(y_treino)

    print (numero_kneighbors, acuracia_teste*100, acuracia_treino*100)

  K   |  %   
1800 54.425000000000004 54.974999999999994
1810 54.50000000000001 55.018750000000004
1820 54.400000000000006 55.1
1830 54.225 55.10625
1840 54.25 55.118750000000006
1850 54.300000000000004 55.0875


In [42]:
classificador = KNeighborsClassifier(n_neighbors=1850, weights='uniform')
feature_selector = RFE(classificador, step=1)
classificadorKNN = feature_selector.fit(x_treino, y_treino)

ValueError: when `importance_getter=='auto'`, the underlying estimator KNeighborsClassifier should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.